# Background

The simulations record the trajectories (a series of x,y,z coordinates, with associated amplitude of each point) of products resulting from collision/reaction between Mg22 and alpha particles in ATTPC.

# User-Desired Settings

The isotope used in this experiment is Mg22.

In [ ]:
ISOTOPE = 'Mg22'

The neural network model requires a fixed number of inputs. Whereas the actual events comprise different number of points, we will select exactly 512 points (may be redundant) as final inputs of each event.

In [ ]:
sample_size = 256

We create a folder named "test" to store the outputs.

In [ ]:
dir_name = './'

# Data Processing

## Import Libraries

In [ ]:
import h5py
import numpy as np
import tqdm
import math
import random
import copy
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from mpl_toolkits import mplot3d

## Import Data
There are two files:
1. "Mg22_alpha.h5" contains two-track events: Mg + alpha --> Mg + alpha
2. "output_digi_HDF_Mg22_Ne20pp_8MeV.h5" contains four-track events: Mg + alpha --> alpha + neon + proton + proton

Each file comprises 10000 events. \
Each event comprises 353-1852 points. \
Each point has 12 entries: x[0], y[1], z[2], time[3], amplitude[4], particleID[5], pointID[6], energy[7] ,energy loss[8], angle[9], mass[10], atomic number[11]. \
The only meaningful entries are x[0], y[1], z[2], amplitude[4].

In [ ]:
def import_data(file, index):
    event_ids = list(file.keys())
    num_of_event = len(event_ids)
    ev_lens = np.zeros(num_of_event, int)
    for i in range(num_of_event):
        event_id = event_ids[i]
        event = file[event_id]
        ev_lens[i] = len(file[event_id])
    evlen_path = dir_name + ISOTOPE + '_sim' + str(index) + '_XYZAPPE_ev_lens'
    np.save(evlen_path, ev_lens)

    data = np.zeros((num_of_event, np.max(ev_lens), 7), float) # XYZAPPE
    for n in tqdm.tqdm(range(num_of_event)):
        event_id = event_ids[n]
        event = file[event_id]
        #converting event into an array
        for i,e in enumerate(event):
            instant = np.array(list(e))
            data[n][i][0:3] = np.array(instant[0:3]) # x,y,z
            data[n][i][3] = np.array(instant[4]) # amplitude
            data[n][i][4] = np.array(instant[5])-1 # particleID--lower index to start at 0
            data[n][i][5] = np.arange(1,np.max(ev_lens)+1)[i] # pointID
            data[n][i][-1] = float(n) # eventID
    data_path = dir_name  + ISOTOPE + '_sim' + str(index) + '_XYZAPPE'
    np.save(data_path, data)
    
    return ev_lens, data

In [ ]:
file1 = h5py.File(dir_name + 'Mg22_alpha.h5', 'r') # 2-track
file2 = h5py.File(dir_name + 'output_digi_HDF_Mg22_Ne20pp_8MeV.h5', 'r') # 4-track
ev_lens1, data1 = import_data(file1, 1)
ev_lens2, data2 = import_data(file2, 2)

When running this notebook the second time, simply reload the data (instead of spending 10 min to repeat the step above).

In [ ]:
ev_lens1 = np.load(dir_name + ISOTOPE + '_sim1_XYZAPPE_ev_lens.npy')
data1 = np.load(dir_name + ISOTOPE + '_sim1_XYZAPPE.npy')
ev_lens2 = np.load(dir_name + ISOTOPE + '_sim2_XYZAPPE_ev_lens.npy')
data2 = np.load(dir_name + ISOTOPE + '_sim2_XYZAPPE.npy')

### Plot Events

In [ ]:
color = ['b', 'y', 'g', 'c', 'k', 'gray']

def plot(evlen_path, data_path, rows, index, noise):
    ev_lens = np.load(evlen_path)
    data = np.load(data_path)
    fig = plt.figure(figsize=(18,rows*4))
    for n in range(rows*5):
        ax = fig.add_subplot(rows, 5, n+1, projection='3d')
        ev_id = int(data[n,0,6])
        ev_len = int(ev_lens[ev_id])
        evt = data[ev_id,:ev_len,:]
        for i,e in enumerate(evt):
            x = e[0] #get x value of instance
            y = e[1] #get y value of instance
            z = e[2] #get z value of instance
            label = int(e[4])
            if index == 1:
                if label == 0: #Mg
                    clr = color[1]
                elif label == 1:  #alpha
                    clr = color[0]
                elif label == 2:  #noise
                    clr = color[4]
                else:  #unknown
                    clr = color[5]
            elif index == 2:
                if label == 0: #alpha
                    clr = color[0]
                elif label == 1: #neon
                    clr = color[1]
                elif label == 2 or label == 3: #proton
                    clr = color[3]
                elif label == 4:  #noise
                    clr = color[4]
                else:  #unknown
                    clr = color[5]
            ax.scatter3D(x, y, z, c = clr, s = 1) 

        ax.set_xlabel('x')
        ax.set_ylabel('y')
        ax.set_zlabel('z')

        A1 = patches.Patch(color=color[0], label = 'alpha')
        A2 = patches.Patch(color=color[3], label = 'proton')
        A3 = patches.Patch(color=color[1], label = 'Mg')
        A4 = patches.Patch(color=color[2], label = 'neon')
        A5 = patches.Patch(color=color[4], label = 'noise')
        
        if (index == 1):
            if (noise == 0):
                plt.legend(handles=[A1, A3], fontsize='small')
            else:
                plt.legend(handles=[A1, A3, A5], fontsize='small')
            plt.suptitle('Mg + alpha --> Mg + alpha', fontsize=25)
        elif (index == 2):
            if (noise == 0):
                plt.legend(handles=[A1, A2, A4], fontsize='small')
            else:
                plt.legend(handles=[A1, A2, A4, A5], fontsize='small')
            plt.suptitle('Mg + alpha --> alpha + neon + proton + proton', fontsize=25)

        plt.title('Event {} \n'.format(n) + str(ev_len) + ' points')
        
    if index == 1 and noise == 0:
        plt.suptitle('2-Track Data', fontsize=25)
    elif index == 1 and noise == 1:
        plt.suptitle('2-Track Data with Noise', fontsize=25)
    elif index == 2 and noise == 0:
        plt.suptitle('4-Track Data', fontsize=25)
    elif index == 2 and noise == 1:
        plt.suptitle('4-Track Data with Noise', fontsize=25)

In [ ]:
def plot_charge(evlen_path, data_path, rows, index, noise):
    ev_lens = np.load(evlen_path)
    data = np.load(data_path)
    fig = plt.figure(figsize=(18,rows*4))
    for n in range(rows*5):
        ax = fig.add_subplot(rows, 5, n+1, projection='3d')
        ev_id = int(data[n,0,6])
        ev_len = int(ev_lens[ev_id])
        evt = data[ev_id,:ev_len,:]
        for i,e in enumerate(evt):
            x = e[0] #get x value of instance
            y = e[1] #get y value of instance
            z = e[2] #get z value of instance
            c = e[3]
            print(c)
            
            ax.scatter3D(x, y, z, c = c, s = 1) 

        ax.set_xlabel('x')
        ax.set_ylabel('y')
        ax.set_zlabel('z')

        

In [ ]:
evlen_path =  dir_name + ISOTOPE + '_sim1_XYZAPPE_ev_lens.npy'
data_path = dir_name + ISOTOPE + '_sim1_XYZAPPE.npy'
plot(evlen_path, data_path, 2, 1, 0)
evlen_path =  dir_name + ISOTOPE + '_sim2_XYZAPPE_ev_lens.npy'
data_path = dir_name + ISOTOPE + '_sim2_XYZAPPE.npy'
plot(evlen_path, data_path, 2, 2, 0)

In [ ]:
evlen_path =  dir_name + ISOTOPE + '_sim1_XYZAPPE_ev_lens.npy'
data_path = dir_name + ISOTOPE + '_sim1_XYZAPPE.npy'
plot_charge(evlen_path, data_path, 2, 1, 0)

## Add Random Noise to Data
We add some (uniformly) random noise to the simulated data so that we could train Model 1 to distinguish noise from real detections.

In [ ]:
def add_noise(ev_lens, data, noise_label, index):
    noise_data = np.zeros((len(ev_lens), np.max(ev_lens)*2, 7), float) # XYZAPPE
    noise_ev_lens = np.zeros(len(ev_lens), int)
    for i in tqdm.tqdm(range(len(ev_lens))):
        ev_len = ev_lens[i]
        noise_ev_lens[i] = ev_len*2
        evt = data[i]
        noise_data[i][0:ev_len] = evt[0:ev_len]
        min = np.zeros(3, float) # xmin, ymin, zmin
        max = np.zeros(3, float) # xmax, ymax, zmax
        for k in range(3):
            try:
                min[k] = np.min(evt[0:ev_len,k])
                max[k] = np.max(evt[0:ev_len,k])
            except ValueError: 
                pass
        for j in range(ev_len):
            pt = j + ev_len
            for k in range(3):
                rand = random.uniform(min[k],max[k])
                noise_data[i][pt][k] = rand
                noise_data[i][pt][4] = noise_label
                noise_data[i][pt][5] = j+ev_len
                noise_data[i][pt][6] = i
        
        evlen_path = dir_name + 'data/' + ISOTOPE + '_sim' + str(index) + '_noise_XYZAPPE_ev_lens'
        data_path = dir_name + 'data/' + ISOTOPE + '_sim' + str(index) + '_noise_XYZAPPE'
    np.save(evlen_path, noise_ev_lens)
    np.save(data_path, noise_data)
    
    return noise_ev_lens, noise_data

In [ ]:
noise_ev_lens1, noise_data1 = add_noise(ev_lens1, data1, 2, 1)
noise_ev_lens2, noise_data2 = add_noise(ev_lens2, data2, 4, 2)

### Plot Events

In [ ]:
evlen_path =  dir_name + ISOTOPE + '_sim1_noise_XYZAPPE_ev_lens.npy'
data_path = dir_name + ISOTOPE + '_sim1_noise_XYZAPPE.npy'
plot(evlen_path, data_path, 2, 1, 1)
evlen_path =  dir_name + ISOTOPE + '_sim2_noise_XYZAPPE_ev_lens.npy'
data_path = dir_name + ISOTOPE + '_sim2_noise_XYZAPPE.npy'
plot(evlen_path, data_path, 2, 2, 1)

## Sample Points

In [ ]:
def sample(ev_lens, data, sample_size, labels_size, index):
    sampled_data = np.zeros((len(ev_lens), sample_size, 8), float) # XYZAPPET
    selected_data = np.zeros((len(ev_lens), sample_size, 8), float) # XYZAPPET

    count = 0
    for i in tqdm.tqdm(range(len(ev_lens))):
        evt = data[i]
        ev_len = ev_lens[i]
        
        unique_labels = np.unique(evt[:,4])
        unique_labels_size = unique_labels.size
        if (unique_labels_size != labels_size or ev_len <= 100):
            continue
    
        particle_id = evt[:ev_len,4]
        label, distr = np.unique(particle_id, return_counts=True)
        shortest = label[np.argmin(distr)]
        shortest_ind = np.argwhere(particle_id == shortest)
        if ev_len == sample_size:    # if array is already preferred length
            sampled_data[i][:,:-1] = data[i][0:ev_len,:]
        else:
            instant = 0
            if shortest_ind.size < sample_size:
                for n in range(shortest_ind.size):    # the first points sampled will be those belonging to the shortest track
                    sampled_data[i,instant,:-1] = data[i,shortest_ind[n],:]
                    instant += 1
                need = sample_size - shortest_ind.size
            else:
                need = sample_size
            random_points = np.random.choice(range(ev_len), need, replace= True if need > ev_len else False)  #choosing the random points to sample
            for r in random_points:
                sampled_data[i,instant,:-1] = data[i,r,:] 
                instant += 1
                    
        unique_labels = np.unique(sampled_data[i,:,4])    # array of unique particleIDs
        unique_labels_size = unique_labels.size       
        if unique_labels_size != labels_size:
            continue

        selected_data[count] = sampled_data[i]
        selected_data[count,0,-1] = labels_size
        
        count += 1
        
    if (labels_size == 2 or labels_size == 4):
        data_path = dir_name + ISOTOPE + '_sim' + str(index) + '_sampled_XYZAPPE'
    else:
        data_path = dir_name + ISOTOPE + '_sim' + str(index) + '_noise_sampled_XYZAPPE'
    np.save(data_path, selected_data[:count, :,:])    
    
    return selected_data[:count, :,:]

In [ ]:
sampled_data1 = sample(ev_lens1, data1, sample_size, 2, 1)
sampled_data2 = sample(ev_lens2, data2, sample_size, 4, 2)

### Plot Events

In [ ]:
evlen_path =  dir_name + ISOTOPE + '_sim1_XYZAPPE_ev_lens.npy'
data_path = dir_name + ISOTOPE + '_sim1_sampled_XYZAPPE.npy'
plot(evlen_path, data_path, 2, 1, 0)
evlen_path =  dir_name + ISOTOPE + '_sim2_XYZAPPE_ev_lens.npy'
data_path = dir_name + ISOTOPE + '_sim2_sampled_XYZAPPE.npy'
plot(evlen_path, data_path, 2, 2, 0)

In [ ]:
evlen_path =  dir_name + ISOTOPE + '_sim1_XYZAPPE_ev_lens.npy'
data_path = dir_name + ISOTOPE + '_sim1_sampled_XYZAPPE.npy'
plot_charge(evlen_path, data_path, 2, 1, 0)

## Create a Dataset Combining 2-track and 4-track Events

In [ ]:
def combine(data1, data2):
    l1 = len(data1)
    l2 = len(data2)
    combined_data = np.zeros((l1+l2, sample_size, 8), float) # XYZAPPET
    combined_data[:l1,:,:] = data1
    combined_data[l1:l1+l2,:,:] = data2
    if (data1[0,0,-1] == 2):
        data_path = dir_name + ISOTOPE + '_sim12_sampled_XYZAPPE'
    else:
        data_path = dir_name + ISOTOPE + '_sim12_noise_sampled_XYZAPPE'
    np.save(data_path, combined_data)
    return combined_data

In [ ]:
combined_data12 = combine(sampled_data1, sampled_data2)

## Get XYZC

In [ ]:
data = np.load(dir_name + ISOTOPE + '_sim12_sampled_XYZAPPE.npy')
new_data = data[:,:, [0,1,2,3]]
data_path = dir_name + ISOTOPE + '_sim12_sampled_XYZC'
np.save(data_path, new_data)

## Pick 10

In [ ]:
np.random.shuffle(sampled_data2)
data = sampled_data2[:10]
new_data = data[:,:, [0,1,2,4]]
data_path = "Mg22_Unpair/" + ISOTOPE + '_sim12_sampled_XYZC_picked'
np.random.shuffle(new_data)
data_scaled = new_data
data_scaled[:,:,3] = np.log10(new_data[:,:,3] + 1e-10)

In [ ]:
for n in range(4):
    mean = np.mean(new_data[:,:,n])
    std = np.std(new_data[:,:,n])
    data_scaled[:,:,n] = (new_data[:,:,n] - mean) / std
np.save(data_path, data_scaled)

In [ ]:
def plot_4d_point_clouds(arr):
    fig = plt.figure(figsize=(10, 10))

    for i in range(len(arr)):
        
        ax = fig.add_subplot(2, 5, i+1, projection='3d') # Creating subplots
        cloud = arr[i]
        # Splitting 4D point cloud into x, y, z coordinates and color
        xs = cloud[:, 0]
        ys = cloud[:, 1]
        zs = cloud[:, 2]
        colors = cloud[:, 3]
        
        # Normalizing color values to range [0,1]
        colors = (colors - np.min(colors)) / (np.max(colors) - np.min(colors))
        
        sc = ax.scatter(xs, ys, zs, c=colors, cmap=plt.cool(), s = 1)
        ax.set_xlim([-2.5, 2.5])
        ax.set_ylim([-2.5, 2.5])
        ax.set_zlim([-2.5, 2.5])
        ax.set_title(f'Point Cloud {i+1}')
        plt.colorbar(sc,shrink = 0.5)

    plt.tight_layout()
    plt.show()

In [ ]:
plot_4d_point_clouds(data_scaled)

## Normalize

In [ ]:
data_path = dir_name + ISOTOPE + '_sim12_sampled_XYZC.npy'
data_sampled = np.load(data_path)
data_scaled = data_sampled

data_scaled[:,:,3] = np.log10(data_sampled[:,:,3] + 1e-10)

for n in range(4):
    mean = np.mean(data_sampled[:,:,n])
    std = np.std(data_sampled[:,:,n])
    data_scaled[:,:,n] = (data_sampled[:,:,n] - mean) / std

data_path = dir_name + ISOTOPE + '_sim12_sampled_scaled_XYZC'
np.save(data_path, data_scaled)

## Create sim'

In [ ]:
selected = [2,4,19,21,22,24,26,30,39,66]
new_data = sampled_data2[selected]
new_data = new_data[:,:, [0,1,2,4]]
print(new_data.shape)

In [ ]:
data_scaled = new_data

data_scaled[:,:,3] = np.log10(new_data[:,:,3] + 1e-10)

for n in range(4):
    mean = np.mean(new_data[:,:,n])
    std = np.std(new_data[:,:,n])
    data_scaled[:,:,n] = (new_data[:,:,n] - mean) / std
plot_4d_point_clouds(data_scaled)

In [ ]:
grids_min = [[0,0,-3],[-3,-3,0]]
grids_max = [[0.5,0.5,3],[3,3,0.5]]
data_path = "Mg22_Unpair/" + ISOTOPE + '_simulated_undeleted'
data_path_del = "Mg22_Unpair/" + ISOTOPE + '_simulated_deleted'
# grids_min = []
# grids_max = []

data = data_scaled
data_deleted = []
for point_cloud in data:
    for grid_min,grid_max in zip(grids_min, grids_max):
        inside_grid = np.all((grid_min <= point_cloud[:,:3]) & (point_cloud[:,:3] <= grid_max), axis=1)
        point_cloud = point_cloud[~inside_grid]
    data_deleted.append(point_cloud)
    
data_deleted = np.array(data_deleted)
plot_4d_point_clouds(data_deleted)


In [ ]:
grids_min = [[0,0,-3],[-3,-3,0]]
grids_max = [[0.5,0.5,3],[3,3,0.5]]
data_path = "Mg22_Unpair/" + ISOTOPE + '_simulated_undeleted'
data_path_del = "Mg22_Unpair/" + ISOTOPE + '_simulated_deleted'
# grids_min = []
# grids_max = []

data = data_scaled
data_deleted = []
for point_cloud in data:
    for grid_min,grid_max in zip(grids_min, grids_max):
        inside_grid = np.all((grid_min <= point_cloud[:,:3]) & (point_cloud[:,:3] <= grid_max), axis=1)
        point_cloud = point_cloud[~inside_grid]
    data_deleted.append(point_cloud)
    
data_deleted = np.array(data_deleted)
plot_4d_point_clouds(data_deleted)


In [ ]:
def sample_4d(data, sample_size=512):
    num_points, dim = data.shape
    assert dim == 4, "Input data should be 4D"

    sampled_data = np.zeros((sample_size, dim))

    if num_points == sample_size:    # if array is already preferred length
        sampled_data = data
    else:
        random_points = np.random.choice(range(num_points), sample_size, replace=True if num_points < sample_size else False)  #choosing the random points to sample
        sampled_data = data[random_points,:]

    return sampled_data

In [ ]:
print(data.shape)
print(data_deleted.shape)
ev_lens = [len(arr) for arr in data_deleted]
print(ev_lens)
data_deleted_512 = np.empty([10,512,4])
for i in range(len(data_deleted)):
    data_deleted_512[i] = sample_4d(data_deleted[i])
print(data_deleted_512.shape)

In [ ]:
plot_4d_point_clouds(data_deleted)

In [ ]:
np.save(data_path, data)
np.save(data_path_del, data_deleted_512)